In [1]:
import time
import random
from datetime import datetime, timedelta

import firebase_admin
from firebase_admin import credentials, db

import pandas as pd
import matplotlib.pyplot as plt


DATABASE_URL = "https://sit-225-5p-default-rtdb.firebaseio.com/Book1"  # <-- replace
SERVICE_KEY = "firebase-key.json"  # path to your service account JSON
PATH = "sensors/DHT22"             # DB path to store the samples
N_SAMPLES = 60                     # how many rows to generate
INTERVAL_S = 10                    # seconds between samples (simulating real time)


# Init Firebase Admin
cred = credentials.Certificate(SERVICE_KEY)
firebase_admin.initialize_app(cred, {"databaseURL": DATABASE_URL})

def generate_sample(ts: datetime):
    """Return one DHT22 sample dict."""
    return {
        "sensor_name": "DHT22",
        "timestamp": ts.isoformat() + "Z",
        "temperature": round(random.uniform(22.0, 28.0), 2),
        "humidity": round(random.uniform(45.0, 65.0), 2),
    }

def upload_samples():
    ref = db.reference(PATH)
    start = datetime.utcnow().replace(microsecond=0)
    for i in range(N_SAMPLES):
        sample = generate_sample(start + timedelta(seconds=i * INTERVAL_S))
        # push creates a unique key under PATH
        ref.push(sample)
        # comment the sleep if you want all data at once
        # time.sleep(INTERVAL_S)
    print(f"Uploaded {N_SAMPLES} samples to /{PATH}")

def download_to_csv_and_plot():
    ref = db.reference(PATH)
    data = ref.get() or {}
    # data is a dict of {autoKey: sample}
    df = pd.DataFrame.from_dict(data, orient="index")
    # sort by timestamp in case
    df = df.sort_values("timestamp")
    csv_path = "dht22_firebase_data.csv"
    img_path = "dht22_firebase_plot.png"
    df.to_csv(csv_path, index=False)

    # Plot
    t = pd.to_datetime(df["timestamp"])
    plt.figure()
    plt.plot(t, df["temperature"], label="Temperature (°C)")
    plt.plot(t, df["humidity"], label="Humidity (%)")
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.title("DHT22 Readings (from Firebase)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(img_path, bbox_inches="tight")
    plt.close()

    print(f"Saved CSV -> {csv_path}")
    print(f"Saved plot -> {img_path}")

if __name__ == "__main__":
    upload_samples()
    download_to_csv_and_plot()


Matplotlib is building the font cache; this may take a moment.
/var/folders/z3/2l40y8_n1hl9vns1b7_vgr1r0000gn/T/ipykernel_5562/1916140163.py:34: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start = datetime.utcnow().replace(microsecond=0)


Uploaded 60 samples to /sensors/DHT22
Saved CSV -> dht22_firebase_data.csv
Saved plot -> dht22_firebase_plot.png
